In [1]:
import collections
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import date
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential
from functools import partial

In [3]:
url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
daily='csse_covid_19_daily_reports/' #+date in the form  01-22-2020.csv
time_serie='csse_covid_19_time_series/' #+confirmed, death,recovered
confirmed_data='time_series_covid19_confirmed_global.csv'
death_data='time_series_covid19_deaths_global.csv'
recovered_data='time_series_covid19_recovered_global.csv'

In [4]:
def make_date(day,month):   #convert a date in (day,month) format to a string to feed in the read_csv function
    day=str(day)
    month=str(month)
    if len(day)==1:
        day='0'+day
    if len(month)==1:
        month='0'+month 
    date=month+'-'+day+'-2020'
    return date
today = str(date.today())
print(today)
a=int(today[5])
b=int(today[6])
c=int(today[8])
d=int(today[9])
today=make_date(a+b,c+d)
print(today)
def add_xday(a,b,c,d,x):
    day=a*10+b
    month=10*c+d
    if month==4 or month==6:
        endmonth=30
    else:
        endmonth=31
    newday=day+x
    if newday>endmonth:
        month+=1
        newday=newday-endmonth
    return make_date(newday,month)

def subdate (date):
    a=date[0]
    b=date[1]
    c=date[3]
    d=date[4]
    return '2020-'+a+b+'-'+c+d
print(subdate(today))
todayplus2=add_xday(a,b,c,d,2)
todayplusweek=add_xday(a,b,c,d,7)


2020-04-12
03-04-2020
2020-03-04


In [17]:
confirmed=pd.read_csv(url+time_serie+confirmed_data)
dead=pd.read_csv(url+time_serie+death_data)
recovered=pd.read_csv(url+time_serie+recovered_data)
country=confirmed[['Province/State','Country/Region']]
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,273,281,299,349,367,423,444,484,521,555
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,277,304,333,361,377,383,400,409,416,433
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,986,1171,1251,1320,1423,1468,1572,1666,1761,1825
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,428,439,466,501,525,545,564,583,601,601
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,8,8,10,14,16,17,19,19,19,19


In [6]:
pop_us=pd.read_csv('pop_us.csv')
pop_us=pop_us[['State','Pop']]


In [7]:
#This cell save the data for the us in a dictionary us_data. Pay attention to adjuste to if statement to
#adjust it at the current date. The value of the dict are dataframe with: first ligne:confirme case, second: dead, 
#third: recovered. the columns are the different dates
us_data={}
total_state=[]
for month in range(1,5):
    for day in range(1,32):
        if month==1 and day<23:
            pass
        elif month==2 and day>29:
            pass
        elif month==4 and day>9:
            pass
        else:
            dat=make_date(day,month)
            #print(dat)
            today=pd.read_csv(url+daily+dat+'.csv')
            if month<3 or (month==3 and day<22):
                st='Province/State'
                country='Country/Region'
            else:
                st='Province_State'
                country='Country_Region'
            for i, state in enumerate(today[st]):
                if today[country].iloc[i]!='US':
                    pass
                else:
                    if state not in total_state:
                        total_state.append(state)
                        us_data[state]=pd.DataFrame()
                        us_data[state][dat]=np.array([today['Confirmed'].iloc[i],today['Deaths'].iloc[i],today['Recovered'].iloc[i]])
                    elif dat not in us_data[state].columns:
                        us_data[state][dat]=np.array([today['Confirmed'].iloc[i],today['Deaths'].iloc[i],today['Recovered'].iloc[i]])
                    else:
                        us_data[state][dat]+=np.array([today['Confirmed'].iloc[i],today['Deaths'].iloc[i],today['Recovered'].iloc[i]])
                        
                    
  


In [8]:
state_us=us_data.keys()
pop_keys=pop_us['State']


# Data Preprocessing 


In [35]:
pop_china=pd.read_csv('pop_china.csv')
print(pop_china)

          Province          Pop
0        Guangdong  104,303,132
1         Shandong  100,063,065
2            Henan   94,023,567
3          Sichuan   80,418,200
4          Jiangsu   78,659,903
5            Hebei   71,854,202
6            Hunan   65,683,722
7            Anhui   59,500,510
8            Hubei   57,237,740
9         Zhejiang   54,426,891
10         Guangxi   46,026,629
11          Yunnan   45,966,239
12         Jiangxi   44,567,475
13        Liaoning   43,746,323
14          Fujian   36,894,216
15         Shaanxi   37,327,378
16    Heilongjiang   38,312,224
17          Shanxi   37,022,111
18         Guizhou   35,806,468
19       Chongqing   28,846,170
20           Jilin   27,462,297
21           Gansu   25,575,254
22  Inner Mongolia   24,706,321
23        Xinjiang   21,813,334
24        Shanghai   23,019,148
25         Beijing   19,612,368
26         Tianjin   12,938,224
27          Hainan    9,261,518
28       Hong Kong    7,061,200
29         Ningxia    6,176,900
30      

In [42]:
X_china={}
for i,land in enumerate(country['Country/Region']):
    if land=='China':
        
        state=confirmed['Province/State'].iloc[i]
        print(state)
        a=np.where(pop_china['Province'] == state)
        print(a)
        a=pop_china['Pop'].iloc[a].values
        a=str(a).replace(',','')
        a=a.replace('[','')
        a=a.replace(']','') 
        a=a.replace("'",'')
        print(a)
        N=float(a)
        print(N)
        print('--')
        length=len(confirmed.iloc[0,4:])
        X=np.zeros((5,length))
        X[0,:]=confirmed.iloc[i,4:]
        X[1,:]=dead.iloc[i,4:]
        X[2,:]=recovered.iloc[i,4:]
        X[3,:]=N*np.ones_like(X[0,:])-X[0,:]-X[1,:]-X[2,:]
        X[4,:]=X[3,:]*X[0,:]
        X=X.reshape(1,-1,5)
        X_china[state]=[]
        X_china[state].append(X[:,:-2,:])
        X_china[state].append(X[:,2:,:3])
        X_china[state].append(X[:,-1,:])
      
        

Anhui
(array([7]),)
59500510
59500510.0
--
Beijing
(array([25]),)
19612368
19612368.0
--
Chongqing
(array([19]),)
28846170
28846170.0
--
Fujian
(array([14]),)
36894216
36894216.0
--
Gansu
(array([21]),)
25575254
25575254.0
--
Guangdong
(array([0]),)
104303132
104303132.0
--
Guangxi
(array([10]),)
46026629
46026629.0
--
Guizhou
(array([18]),)
35806468
35806468.0
--
Hainan
(array([27]),)
9261518
9261518.0
--
Hebei
(array([5]),)
71854202
71854202.0
--
Heilongjiang
(array([16]),)
38312224
38312224.0
--
Henan
(array([2]),)
94023567
94023567.0
--
Hong Kong
(array([28]),)
7061200
7061200.0
--
Hubei
(array([8]),)
57237740
57237740.0
--
Hunan
(array([6]),)
65683722
65683722.0
--
Inner Mongolia
(array([22]),)
24706321
24706321.0
--
Jiangsu
(array([4]),)
78659903
78659903.0
--
Jiangxi
(array([12]),)
44567475
44567475.0
--
Jilin
(array([20]),)
27462297
27462297.0
--
Liaoning
(array([13]),)
43746323
43746323.0
--
Macau
(array([32]),)
552503
552503.0
--
Ningxia
(array([29]),)
6176900
6176900.0
--
Qi

In [44]:
pred=pd.DataFrame()

# RNN  training for China


In [47]:
shape=np.shape(X_china['Anhui'][0])[1]
print(shape)
model_china=Sequential()
model_china.add(LSTM(units=30,return_sequences=True,input_shape=(shape,5)))
model_china.add(Dropout(0.25))
model_china.add(LSTM(units=30,return_sequences=True))
model_china.add(Dropout(0.25))
model_china.add(LSTM(units=30,return_sequences=True))
model_china.add(Dropout(0.25))
model_china.add(Dense(units=3, activation=partial(tf.nn.leaky_relu, alpha=0.01)))
model_china.summary()
model_china.compile(optimizer='adam',loss='mean_squared_error')

for state in X_china.keys():
    print(np.shape(X_china[state][2]))
    print(state)
    #print(X_china[state][0])
    model_china.fit(X_china[state][0],X_china[state][1],epochs=20)
    pred[state]=model_china.predict(X_china[state][2])
    break


79
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 79, 30)            4320      
_________________________________________________________________
dropout_21 (Dropout)         (None, 79, 30)            0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 79, 30)            7320      
_________________________________________________________________
dropout_22 (Dropout)         (None, 79, 30)            0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 79, 30)            7320      
_________________________________________________________________
dropout_23 (Dropout)         (None, 79, 30)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 79, 3)         

ValueError: Error when checking input: expected lstm_21_input to have 3 dimensions, but got array with shape (1, 5)

In [ ]:
a=model_china.predict(X_china['Beijing'][0])
X=X_china['Beijing'][1][0,:,0]
print(X)
times=np.arange(len(X))
plt.figure()
plt.plot(times,X)
plt.show()

In [ ]:

for state in X_china.keys():
    plt.figure()
    X=X_china[state][0][0,:,0]
    time=np.arange(len(X))
    plt.plot(time,X,label=state)
    plt.show()
print(X)